# Demonstration of how to calc rho from ERA5 data

Useful demo code showing what fields are required, how to call ERA5 calc etc.

* Move to templates.

In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import ndimage
import xarray as xr

from remake import load_remake

from mcs_prime import PATHS
%matplotlib inline

In [2]:
# Load some ERA5 u, v, q data.
year = 2020
month = 1
day = 11
e5datadir = PATHS['era5dir'] / f'data/oper/an_ml/{year}/{month:02d}/{day:02d}'

In [3]:
h = 6

e5time = dt.datetime(year, month, day, h, 0)
paths = [e5datadir / (f'ecmwf-era5_oper_an_ml_{t.year}{t.month:02d}{t.day:02d}'
                      f'{t.hour:02d}00.{v}.nc')
         for t in [e5time, e5time + dt.timedelta(hours=1)]
         for v in ['t', 'q', 'lnsp']]
paths

[PosixPath('/badc/ecmwf-era5/data/oper/an_ml/2020/01/11/ecmwf-era5_oper_an_ml_202001110600.t.nc'),
 PosixPath('/badc/ecmwf-era5/data/oper/an_ml/2020/01/11/ecmwf-era5_oper_an_ml_202001110600.q.nc'),
 PosixPath('/badc/ecmwf-era5/data/oper/an_ml/2020/01/11/ecmwf-era5_oper_an_ml_202001110600.lnsp.nc'),
 PosixPath('/badc/ecmwf-era5/data/oper/an_ml/2020/01/11/ecmwf-era5_oper_an_ml_202001110700.t.nc'),
 PosixPath('/badc/ecmwf-era5/data/oper/an_ml/2020/01/11/ecmwf-era5_oper_an_ml_202001110700.q.nc'),
 PosixPath('/badc/ecmwf-era5/data/oper/an_ml/2020/01/11/ecmwf-era5_oper_an_ml_202001110700.lnsp.nc')]

In [4]:
e5 = xr.open_mfdataset(paths).sel(latitude=slice(60, -60)).mean(dim='time').load()
T = e5.t
q = e5.q
lnsp = e5.lnsp

df_ecmwf = pd.read_csv('/gws/nopw/j04/mcs_prime/mmuetz/data/ERA5/ERA5_L137_model_levels_table.csv')

df_ecmwf
# df_ecmwf = pd.read_csv(PATHS['datadir'] / 'ecmwf_L137_model_level_definitions.csv')

,n,a [Pa],b,ph [hPa],pf [hPa],Geopotential Altitude [m],Geometric Altitude [m],Temperature [K],Density [kg/m^3]
0,0,0.000000,0.000000,0.0000,-,-,-,-,-
1,1,2.000365,0.000000,0.0200,0.0100,79301.79,80301.65,198.05,0.000018
2,2,3.102241,0.000000,0.0310,0.0255,73721.58,74584.91,209.21,0.000042
3,3,4.666084,0.000000,0.0467,0.0388,71115.75,71918.79,214.42,0.000063
4,4,6.827977,0.000000,0.0683,0.0575,68618.43,69365.77,221.32,0.000090
...,...,...,...,...,...,...,...,...,...
133,133,62.781250,0.988500,1002.2250,1000.5165,106.54,106.54,287.46,1.212498
134,134,22.835938,0.991984,1005.3562,1003.7906,79.04,79.04,287.64,1.215710
135,135,3.757813,0.995003,1008.2239,1006.7900,53.92,53.92,287.80,1.218650
136,136,0.000000,0.997630,1010.8487,1009.5363,30.96,30.96,287.95,1.221341


In [5]:
sp = np.exp(lnsp).values  # pressure in Pa.
a = df_ecmwf['a [Pa]'].values  # a now in Pa.
b = df_ecmwf.b.values

In [6]:
# Broadcasting to correctly calc 3D pressure field.
p_half = a[:, None, None] + b[:, None, None] * sp[None, :, :]
p = (p_half[:-1] + p_half[1:]) / 2

In [7]:
p.shape

(137, 481, 1440)

In [8]:
Tv_approx = T * (1 + 0.608 * q)

In [9]:
import metpy.calc as mpcalc
from metpy.units import units

Tv = mpcalc.virtual_temperature(T.values * units.K, q.values).magnitude
Tv

array([[[216.40038, 216.64757, 216.88396, ..., 215.8567 , 215.97914,
         216.16756],
        [216.15077, 216.38716, 216.62114, ..., 215.71391, 215.78955,
         215.94078],
        [215.87718, 216.09917, 216.32594, ..., 215.54834, 215.57957,
         215.6948 ],
        ...,
        [167.15086, 167.14125, 167.12924, ..., 167.15932, 167.15929,
         167.15688],
        [166.96246, 166.95287, 166.94084, ..., 166.9745 , 166.97209,
         166.96849],
        [166.76328, 166.75487, 166.74287, ..., 166.7765 , 166.7741 ,
         166.76929]],

       [[222.92029, 223.0271 , 223.17587, ..., 222.93582, 222.87704,
         222.86867],
        [223.02826, 223.12067, 223.26346, ..., 223.12419, 223.03423,
         222.99825],
        [223.16264, 223.23825, 223.36664, ..., 223.34138, 223.2202 ,
         223.15663],
        ...,
        [175.78773, 175.73735, 175.68936, ..., 175.94135, 175.88976,
         175.83815],
        [175.65337, 175.60417, 175.55617, ..., 175.80336, 175.75296,
   

In [10]:
Rd = 287

rho = p / (Rd * Tv)

In [11]:
rho

array([[[1.61042074e-05, 1.60858324e-05, 1.60682999e-05, ...,
         1.61447679e-05, 1.61356163e-05, 1.61215514e-05],
        [1.61228031e-05, 1.61051900e-05, 1.60877952e-05, ...,
         1.61554548e-05, 1.61497923e-05, 1.61384822e-05],
        [1.61432370e-05, 1.61266538e-05, 1.61097478e-05, ...,
         1.61678646e-05, 1.61655219e-05, 1.61568871e-05],
        ...,
        [2.08491685e-05, 2.08503671e-05, 2.08518648e-05, ...,
         2.08481143e-05, 2.08481177e-05, 2.08484181e-05],
        [2.08726944e-05, 2.08738941e-05, 2.08753968e-05, ...,
         2.08711887e-05, 2.08714898e-05, 2.08719407e-05],
        [2.08976259e-05, 2.08986784e-05, 2.09001829e-05, ...,
         2.08959682e-05, 2.08962701e-05, 2.08968721e-05]],

       [[3.98777419e-05, 3.98586429e-05, 3.98320716e-05, ...,
         3.98749616e-05, 3.98854787e-05, 3.98869767e-05],
        [3.98584362e-05, 3.98419269e-05, 3.98164458e-05, ...,
         3.98412974e-05, 3.98573684e-05, 3.98638004e-05],
        [3.98344329e-05, 

In [12]:
from era5_calc import ERA5Calc
e5calc = ERA5Calc('/gws/nopw/j04/mcs_prime/mmuetz/data/ERA5/ERA5_L137_model_levels_table.csv')

In [13]:
p2 = e5calc.calc_pressure(lnsp.values)
Tv2 = e5calc.calc_Tv(T.values, q.values)
rho2 = e5calc.calc_rho(p, Tv)

In [16]:
(rho == rho2).all()

True

In [17]:
rho.shape

(137, 481, 1440)

In [18]:
rho[-1].mean()

1.1934438547698818

In [19]:
rho_standard = e5calc.df_ecmwf['Density [kg/m^3]'].values[1:].astype(float)


In [20]:
rho_standard[-1]

1.223803

In [21]:
rho[-1]

array([[1.21712132, 1.21733697, 1.21798617, ..., 1.21662355, 1.21722909,
        1.21728691],
       [1.21721695, 1.21750612, 1.21823669, ..., 1.21672981, 1.21702672,
        1.21716745],
       [1.21757859, 1.21812216, 1.21882833, ..., 1.2161486 , 1.21638562,
        1.2169576 ],
       ...,
       [1.26298537, 1.26327834, 1.2634687 , ..., 1.2618326 , 1.26215   ,
        1.26256857],
       [1.26331798, 1.26348179, 1.26354157, ..., 1.26269248, 1.26289405,
        1.26311067],
       [1.26324697, 1.26319399, 1.26306746, ..., 1.26329984, 1.26324745,
        1.26322363]])